In [19]:
import requests
from dotenv import load_dotenv
import os
import json
import base64   
import pandas as pd
import librosa
import numpy as np
import lyricsgenius
import langdetect
import re
import string
import tempfile

import pandas as pd
pd.set_option('display.max_colwidth', None)
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
import scipy.sparse as sp
import numpy as np
import random
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import time
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [3]:
lastfm_api_key = "97d5a64d5ba4a8bc580b752ceff3b87f"
lastfm_secret = "35175090bd61f6f16ac607bd26e5b1de"

In [8]:
from collections import deque, defaultdict
import requests
base_url = 'http://ws.audioscrobbler.com/2.0/'
def get_lastfm_friends_loved_tracks(start_username, api_key, min_users=1000, tracks_per_user=50, min_songs=5000):
    discovered_users = set([start_username])
    queue = deque([start_username])
    users_loved_tracks = defaultdict(list)  # Use defaultdict to easily append tracks
    unique_songs = set()  # Track unique songs to ensure we meet the min_songs requirement

    while queue and (len(discovered_users) < min_users or len(unique_songs) < min_songs):
        current_user = queue.popleft()

        # Fetch loved tracks for the current user
        loved_tracks_payload = {'method': 'user.getlovedtracks', 'user': current_user, 'api_key': api_key, 'format': 'json', 'limit': tracks_per_user}
        loved_tracks_response = requests.get(base_url, params=loved_tracks_payload)
        loved_tracks_data = loved_tracks_response.json()

        if 'error' not in loved_tracks_data:
            loved_tracks = loved_tracks_data.get('lovedtracks', {}).get('track', [])
            for track in loved_tracks:
                track_name = track['name']
                artist_name = track['artist']['name']
                song_identifier = f"{artist_name} - {track_name}"
                
                if song_identifier not in unique_songs:
                    unique_songs.add(song_identifier)
                
                users_loved_tracks[current_user].append(song_identifier)

        # Fetch friends only if we need more users
        if len(discovered_users) < min_users:
            friends_payload = {'method': 'user.getfriends', 'user': current_user, 'api_key': api_key, 'format': 'json'}
            friends_response = requests.get(base_url, params=friends_payload)
            friends_data = friends_response.json()

            if 'error' not in friends_data:
                friends = friends_data.get('friends', {}).get('user', [])
                for friend in friends:
                    friend_name = friend['name']
                    if friend_name not in discovered_users:
                        discovered_users.add(friend_name)
                        queue.append(friend_name)

        print(f"Collected {len(discovered_users)} users and {len(unique_songs)} unique songs so far...")

    return users_loved_tracks, unique_songs

# Use this function to collect your data
users_loved_tracks, unique_songs = get_lastfm_friends_loved_tracks("Bans77", lastfm_api_key, min_users=1000, tracks_per_user=50, min_songs=5000)


Collected 24 users and 50 unique songs so far...
Collected 27 users and 88 unique songs so far...
Collected 69 users and 89 unique songs so far...
Collected 117 users and 139 unique songs so far...
Collected 165 users and 189 unique songs so far...
Collected 181 users and 239 unique songs so far...
Collected 229 users and 289 unique songs so far...
Collected 238 users and 339 unique songs so far...
Collected 287 users and 389 unique songs so far...
Collected 306 users and 389 unique songs so far...
Collected 355 users and 439 unique songs so far...
Collected 405 users and 489 unique songs so far...
Collected 405 users and 489 unique songs so far...
Collected 428 users and 489 unique songs so far...
Collected 440 users and 489 unique songs so far...
Collected 441 users and 489 unique songs so far...
Collected 442 users and 489 unique songs so far...
Collected 492 users and 489 unique songs so far...
Collected 492 users and 489 unique songs so far...
Collected 539 users and 539 unique so

In [18]:
users_loved_tracks.values()

for k in users_loved_tracks.values():
    print(k)

['beabadoobee - Glue Song (feat. Clairo)', 'Phoenix - After Midnight (feat. Clairo)', 'Clairo feat. Rejjie Snow - Hello?', 'Clairo feat. Danny L Harle - B.O.M.D.', 'Clairo - Amoeba (Sirius XM Session)', 'Clairo - Reaper (Sirius XM Session)', 'Clairo - Zinnias (Sirius XM Session)', 'Clairo - Clairo - Flamin_ Hot Cheetos', 'Clairo - Forecastle Festival 2022', 'Clairo - WASTE (BH cover)', 'Clairo - Reaper Live', 'Clairo - libra - demo', 'Clairo - Bubble Gum(MP3_128K)', 'Clairo - Blouse // Español', 'Clairo - no name 004- 20 years instagram live', 'Clairo - Zinnias Live', "Clairo - Amoeba (22')", 'Clairo - Bags (Live World Cafe)', 'Clairo - Bobby Bell Holds the Plate with Two Hands [feat. Clairo]', 'Clairo - Bags (Live Sirius XM Performance)', 'Clairo - Sovia', 'Clairo - Bags - Live', 'Clairo - Bags [LIVE @ SiriusXM]', 'Clairo - Pretty Girl - SLOWED', 'Clairo - favoritism 1)', 'Clairo - Clairo - How Did I Ever (Subtítulos en español) [Lyrics]', 'Clairo - "Bags"', 'Clairo - pink  blue', 'Cl